In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd 
import matplotlib.pyplot as plt

In [ ]:
sms = pd.read_csv("/kaggle/input/sms-spam-collection-dataset/spam.csv", encoding = "ISO-8859-1", usecols=[0,1], skiprows=1,
                  names=["label", "message"])

In [ ]:
sms.head()

In [ ]:
sms.info(
)

In [ ]:
sms.isnull().sum()

In [ ]:
sms['label'] = sms['label'].apply(lambda x: 1 if x == 'spam' else 0)
sms['label']=sms['label'].astype(int)

In [ ]:
# check duplications 
sms.duplicated().sum(),sms.shape

In [ ]:
#delete duplicated values 
sms=sms.drop_duplicates(keep='first')
sms.shape

## Data Analysis

In [ ]:
sms['label']

In [ ]:
sms['label'].value_counts()

In [ ]:
plt.pie(sms['label'].value_counts(),labels=['Not Spam','Spam'], autopct='%1.1f%%')
plt.title('Distribution of Spam vs. Not Spam')
plt.show()

## Convert text to feature 

In [ ]:
import nltk 
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize


In [ ]:
sms['num_char']=sms['message'].apply(len)
sms.head()

In [ ]:
sms['num_words']=sms['message'].apply(lambda x : len(word_tokenize(x)))
sms.head()

In [ ]:
sms['num_sent']=sms['message'].apply(lambda x : len(sent_tokenize(x)))
sms.head()

## Data Analysis for Messages

In [ ]:
sms[['num_char','num_words','num_sent']]

In [ ]:
# for all message
sms[['num_char','num_words','num_sent']].describe()

In [ ]:
# for spam message
sms[sms['label']==1][['num_char','num_words','num_sent']].describe()

In [ ]:
# for not spam message
sms[sms['label']==0][['num_char','num_words','num_sent']].describe()

In [ ]:
import seaborn as sns 
sns.histplot(sms[sms['label'] == 0]['num_char'], color='blue')
sns.histplot(sms[sms['label'] == 1]['num_char'], color='red')


In [ ]:
import seaborn as sns 
sns.histplot(sms[sms['label'] == 0]['num_words'], color='blue')
sns.histplot(sms[sms['label'] == 1]['num_words'], color='red')


In [ ]:
import seaborn as sns 
sns.histplot(sms[sms['label'] == 0]['num_sent'], color='blue')
sns.histplot(sms[sms['label'] == 1]['num_sent'], color='red')


In [ ]:
sns.pairplot(sms,hue='label')

In [ ]:
sns.heatmap(sms[['label','num_char','num_sent','num_words']].corr(),annot=True)

## Data Preprocessing

* Lower case
* Tokenization
* Removing special characters
* Removing stop words and punctuation
* Stemming

In [ ]:
import re
from nltk.corpus import stopwords
import string 
punctuation= string.punctuation
nltk.download('punkt')
#!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~



nltk.download('stopwords')
stop_words = stopwords.words('english')
#['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
# going -> go  ,sleeps ->sleep 



def text_transform(x):
  x=x.lower()
  x = nltk.word_tokenize(x)
  x=[word for word in x if (word not in stop_words) and (word.isalnum()) and (word not in punctuation)]
  x = [stemmer.stem(word) for word in x]

  return " ".join(x)


text_transform("HI HOW ARE YOU ? !,  he goes to sleep   ")

In [ ]:
(text_transform(sms['message'][0]))

In [ ]:
(sms['message'][0])

In [ ]:
sms['text_transformed']=sms['message'].apply(text_transform)
sms.head()

In [ ]:
# word cloud 
import matplotlib.pyplot as plt
from wordcloud import WordCloud


In [ ]:
#spam 
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(   
    
    (sms[sms['label']==1]['text_transformed']).str.cat(sep=" ")
)

# Display the word cloud using Matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

In [ ]:
#spam 
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(   
    
    (sms[sms['label']==0]['text_transformed']).str.cat(sep=" ")
)

# Display the word cloud using Matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.show()

## Model Building

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer


In [ ]:
cv=CountVectorizer()
X=cv.fit_transform(sms['text_transformed']).toarray()
# 6708 feature 
X.shape, sms['text_transformed'].shape

In [ ]:
y=sms['label'].values

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train,y_train)
nb_pred = nb_classifier.predict(X_test)
nb_accuracy = accuracy_score(y_test, nb_pred)
nb_report = classification_report(y_test, nb_pred)
nb_accuracy

In [ ]:
from sklearn.linear_model import LogisticRegression
lg = LogisticRegression(C=20.0,max_iter=1000)
lg.fit(X_train,y_train)
lg_pred = lg.predict(X_test)
lg_accuracy = accuracy_score(y_test, lg_pred)
lg_report = classification_report(y_test, lg_pred)
lg_accuracy

In [ ]:
from sklearn.svm import SVC
sv = SVC(C=10.0)
sv.fit(X_train,y_train)
sv_pred = sv.predict(X_test)
sv_accuracy = accuracy_score(y_test, sv_pred)
sv_report = classification_report(y_test, sv_pred)
sv_accuracy

In [ ]:
print("Naive Bayes Classifier:")
print(f"Accuracy: {nb_accuracy:.2f}")
print("Classification Report:\n", nb_report)

print("\nLogistic Regression Classifier:")
print(f"Accuracy: {lg_accuracy:.2f}")
print("Classification Report:\n", lg_report)

print("\nSupport Vector Machine Classifier:")
print(f"Accuracy: {sv_accuracy:.2f}")
print("Classification Report:\n", sv_report)